In [ ]:
!pip install loralib

In [1]:
import os
from gpt import GPT
from configs import get_configs
from tqdm import tqdm
import torch
import tiktoken
import json
import csv


ModuleNotFoundError: No module named 'torch'

In [ ]:
cfg = get_configs("gpt2-medium")
# model = GPT.from_checkpoint(cfg, ckpt_path='sft_sft_experiment_202312141343_final.pt')
model = GPT.from_checkpoint(cfg, ckpt_path='C:/Users/Ragal/Downloads/sft_sft_experiment_202312181738_final.pt')

# model.eval()
# model.generate()

In [ ]:
def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode


def generate_gpt2(model, prompt, device):
    model.eval()
    model.to(device)
    max_new_tokens = 200
    temperature = 0.9
    top_k = 400
    x, decode = prepare_gpt2_input(prompt, device)

    y = model.generate(x,
                       max_new_tokens,
                       temperature=temperature,
                       top_k=top_k)

    res = decode(y[0].cpu().tolist())
    end = res.find("<|endoftext|>")
    if end > 0:
        return res[:end]
    else:
        return res

In [ ]:
# with open("prompts.csv") as fp:
#     reader = csv.DictReader(fp)
#     prompts = [row["prompt"] for row in reader]

In [3]:
prompts = 'Who asked?'
prompts

'Who asked?'

In [ ]:
# VPN might cause network error
print("Run inference")
if os.path.exists("Response.json"):
    with open("Response.json") as fp:
        Response = json.load(fp)
else:
    device = "cuda" # cpu very slow! use 'cuda' if you have 8g memory
    cfg = get_configs("gpt2-medium")
    with torch.inference_mode():
        gpt_vanilla = GPT.from_pretrained(cfg)
        gpt_sft = model
        Response = []
        for prompt in tqdm(prompts):
            Response.append({
                "vanilla": generate_gpt2(gpt_vanilla, f"Human: {prompt}\n\nAssistant: ", device)[
                               len(f"Human: {prompt}\n\nAssistant: "):],
                "sft": generate_gpt2(gpt_sft, f"Human: {prompt}\n\nAssistant: ", device)[
                           len(f"Human: {prompt}\n\nAssistant: "):],
                "prompt": prompt
                })
            with open("Response.json", "w") as fp:
                json.dump(Response, fp)

In [ ]:
base_res = Response['vanilla']
sft_res = Response['sft']
question = Response['prompt']

In [ ]:
prompts

In [ ]:
import torch
print("Torch version:",torch.__version__)

In [ ]:
torch.cuda.is_available()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
reward_name = "OpenAssistant/reward-model-deberta-v3-large-v2"
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name)
inputs1 = tokenizer(prompt, base_res, return_tensors='pt')
score1 = rank_model(**inputs1).logits[0].cpu().detach()
inputs2 = tokenizer(prompt, sft_res, return_tensors='pt')
score2 = rank_model(**inputs2).logits[0].cpu().detach()
print('Score of the basic model is {}, while the score of the sft model is {}'.format(score1, score2))